## Библиотеки

In [1]:
import os
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from bs4 import BeautifulSoup as bs

import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Создание таблиц с нужными данными

### Функции:

In [1]:
DESKTOP_USER_NAME = "Admin" # имя пользователя на компьютере

In [3]:
def line_to_dict(split_Line):

    line_dict = {}
    for part in split_Line:
        key, value = part.split(":")
        if value == ' true' or value == ' false':
            value = (value == ' true')
        elif value[1:].isnumeric():
            value = int(value[1:])
        elif key == '"sameSite"':
            if value[1:] == '"lax"' or value[1:] == '"strict"':
                value = value[1:].title()
                value = value[value.find('"') + 1 : value.rfind('"')]
            else:
                value = "None"
        else:
            value = value[value.find('"') + 1 : value.rfind('"')]
        key = key[key.find('"') + 1 : key.rfind('"')]
        line_dict[key] = value

    return line_dict

def convert(content) :
    os.remove("cookies.json")
    content = content.split("}")
    CON = []
    for line in content:
        norm = line[line.find('"') - 1 : line.rfind("\n")]
        norm = norm.split(",")
        norm = [x.strip() for x in norm]
        CON.append(norm)
    
    CON = CON[:-1]
    CON = [line_to_dict(x) for x in CON]
    
    # Output JSON 
    with open("cookies.json", 'w+') as fout:
        json.dump(CON, fout)
        
def clear(X):
    if X.rfind("<") > X.find(">"):
        X_clear = X[X.find(">") + 1: X.rfind("<")].strip()
    else:
        X_clear = X[: X.rfind("<")].strip()
    if X_clear.find(">") != -1:
        return clear(X_clear)
    else:
        return X_clear

def to_time(time):
    mas = time.split(" ")
    mas = mas[:-1]
    time = mas[0] + " " + mas[1]
    return pd.to_datetime(time, format = "%d.%m.%Y %H:%M")

def sopostav(A, B):
    B = B[:-2] + [A[-5], A[-3]] + B[-2:]
    B[0] = pd.to_datetime(B[0], format = "%d.%m.%Y")
    B[-1] = to_time(B[-1])
    B[-2] = to_time(B[-2])
    
    for i in range(len(B)):
        if type(B[i]) == str:
            if B[i].isnumeric():
                B[i] = int(B[i])
    B[2] = str(B[2])
    
    A = A[:-5] + [A[-4]] + A[-2:]
    assert len(A) == len(B)
    dic = {}
    for i in range(len(A)):
        dic[A[i]] = B[i]
        if B[i] == '':
            dic[A[i]] = np.nan
    return dic

def help_sopostav(A, B):
    B = B[:-1] + [A[-4], A[-2]] + B[-1:]
    B[0] = pd.to_datetime(B[0], format = "%d.%m.%Y")
    B[-1] = to_time(B[-1])
    B = B + [B[-1]]
    
    for i in range(len(B)):
        if type(B[i]) == str:
            if B[i].isnumeric():
                B[i] = int(B[i])
    B[2] = str(B[2])
    
    A = A[:-4] + [A[-3]] + ['Дата и время начала проведения аукциона', 'Дата и время окончания проведения аукциона']
    assert len(A) == len(B)
    dic = {}
    for i in range(len(A)):
        dic[A[i]] = B[i]
        if B[i] == '':
            dic[A[i]] = np.nan
    return dic

def all_you_need(X):
    massive = X.split(" ")
    massive = [a.strip() for a in massive]
    return " ".join([x for x in massive if x != ""])

def append_zakupka(df_zakupki, soup, flag = True):
    
    global zakupka
    global first_table
    global second_table
    
    everything_important = soup.find_all('table', {"class" : "dt"})
    #print(everything_important)
    first = everything_important[2]
    
    title = clear(str(soup.find_all('h2')[1]))
    if title == "Протокол рассмотрения единственной заявки на электронный аукцион":
        return help_append_zakupka(df_zakupki, soup, flag)
    
    if title == "Изменение протокола подведения итогов электронного аукциона":
        if flag:
            return df_zakupki
        else:
            return df_zakupki.to_dict()
    
    
    everything = first.find_all('td', {"class" : "fr"})
    important = first.find_all('td', {"class" : ""})
    everything, important = [str(x) for x in everything], [str(x) for x in important]
    everything, important = [clear(x) for x in everything], [clear(x) for x in important]
    #print(everything)
    if everything[0] == 'Номер протокола':
        everything, important = everything[1:], important[1:]
    dc_f = sopostav(everything, important)
    
    second = everything_important[4]
    third = [all_you_need(clear(str(x).replace("<br/>", ""))) for x in second.find_all('td', {"class" : "th"})]
    
    dc_s = {x: [] for x in list(second_table.keys())}
    names = list(dc_s.keys())
    #basic_names = names[:7]
    
    second = [str(x) for x in second.find_all('td', {"class" : ""})]
    
    #print(third)
    FLAG = not("Защищенный номер" in third)
    #print(flag)
    
    #second = [second[0]] + ["></td>"] + second[1:]
    
    if FLAG:
        LEN = len(second)
        for i in range(LEN//6):
            second = second[:7*i + 1] + ["></td>"] + second[7*i + 1:]
    #print(second, len(second))

    for i, x in enumerate(second):
        mas = dc_s[names[i % 7]]
        #print(names[i % 7])
        if x.find("<br/>") != -1:
            new = x[x.find(">") + 1: x.find("<br/>")]
        else:
            new = x[x.find(">") + 1: x.find("</td>")]

        if new == '':
            new = np.nan
        elif names[i % 7] in ["Защищенный номер"]:
            new = int(new)
        elif names[i % 7] in ["Предложение о цене контракта"]:
            if new == "-":
                new = 0.0
            new = float(new)
        elif names[i % 7] in ["Дата и время подачи заявки", "Дата и время подачи ценового предложения"]:
            new = pd.to_datetime(new, format = "%d.%m.%Y %H:%M:%S")
        mas.append(new)
        dc_s[names[i % 7]] = mas
    
    mas_winner = list(dc_s["Предложение о цене контракта"])
    #print(mas_winner)
    mas_approved = list(dc_s["Решение комиссии о соответствии или отклонении"])
    mas_date = list(dc_s["Дата и время подачи ценового предложения"])
    if len([x for j, x in enumerate(mas_winner) if (mas_approved[j] == "Соответствует")]) == 0:
        N = zakupka.copy()
        df_zakupki = pd.concat([df_zakupki, pd.DataFrame(N)], ignore_index=True)
        if flag:
            return df_zakupki
        else:
            return N
        
    win = min([x for j, x in enumerate(mas_winner) if (mas_approved[j] == "Соответствует")])
    datetime = min([x for j, x in enumerate(mas_date) if (mas_winner[j] == win) and (mas_approved[j] == "Соответствует")])
    mas_bool = [((x == win) and (mas_approved[j] == "Соответствует") and (mas_date[j] == datetime)) for j, x in enumerate(mas_winner)]
    dc_s["Победитель"] = mas_bool
    
    dc_s["Количество участников"] = [(len(second)//7)] * (len(second)//7)
    
    #print(dc_s)
    
    N = zakupka.copy()
    for x in list(N.keys()):
        if x in list(first_table.keys()):
            mas = [dc_f[x]]* (len(second)//7)
            N[x] = mas
        if x in list(second_table.keys()):
            N[x] = dc_s[x]
    
    df_zakupki = pd.concat([df_zakupki, pd.DataFrame(N)], ignore_index=True)
    
    if flag:
        return df_zakupki
    else:
        return N

def help_append_zakupka(df_zakupki, soup, flag = True):
    
    global zakupka
    global first_table
    global second_table
    
    everything_important = soup.find_all('table', {"class" : "dt"})
    first = everything_important[2]

    title = clear(str(soup.find_all('h2')[1]))

    everything = first.find_all('td', {"class" : "fr"})
    important = first.find_all('td', {"class" : ""})
    everything, important = [str(x) for x in everything], [str(x) for x in important]
    everything, important = [clear(x) for x in everything], [clear(x) for x in important]
    if everything[0] == 'Номер протокола':
        everything, important = everything[1:], important[1:]

    dc_f = help_sopostav(everything, important)

    second = everything_important[4]
    
    third = [all_you_need(clear(str(x).replace("<br/>", ""))) for x in second.find_all('td', {"class" : "th"})]
    
    dc_s = {x: [] for x in list(second_table.keys())}
    names = list(dc_s.keys())
    #basic_names = names[:7]
    
    second = [str(x) for x in second.find_all('td', {"class" : ""})]

    FLAG = not("Защищенный номер" in third)
    
    if FLAG:
        LEN = len(second)
        for i in range(LEN//6):
            second = second[:7*i + 1] + ["></td>"] + second[7*i + 1:]

    for i, x in enumerate(second):
        mas = dc_s[names[i % 7]]
        #print(names[i % 7])
        if x.find("<br/>") != -1:
            new = x[x.find(">") + 1: x.find("<br/>")]
        else:
            new = x[x.find(">") + 1: x.find("</td>")]

        if new == '':
            new = np.nan
        elif names[i % 7] in ["Защищенный номер"]:
            new = int(new)
        elif names[i % 7] in ["Предложение о цене контракта"]:
            if new == "-":
                new = 0.0
            new = float(new)
        elif names[i % 7] in ["Дата и время подачи заявки", "Дата и время подачи ценового предложения"]:
            new = pd.to_datetime(new, format = "%d.%m.%Y %H:%M:%S")
        mas.append(new)
        dc_s[names[i % 7]] = mas
    
    mas_winner = list(dc_s["Предложение о цене контракта"])
    #print(mas_winner)
    mas_approved = list(dc_s["Решение комиссии о соответствии или отклонении"])
    mas_date = list(dc_s["Дата и время подачи ценового предложения"])
    win = min(mas_winner)
    datetime = min([x for j, x in enumerate(mas_date) if mas_winner[j] == win])
    mas_bool = [True] #[((x == win) and (mas_approved[j] == "Соответствует") and (mas_date[j] == datetime)) for j, x in enumerate(mas_winner)]
    dc_s["Победитель"] = mas_bool
    
    dc_s["Количество участников"] = [(len(second)//7)] * (len(second)//7)
    
    #print(dc_s)
    
    N = zakupka.copy()
    for x in list(N.keys()):
        if x in list(first_table.keys()):
            mas = [dc_f[x]]* (len(second)//7)
            N[x] = mas
        if x in list(second_table.keys()):
            N[x] = dc_s[x]
    
    df_zakupki = pd.concat([df_zakupki, pd.DataFrame(N)], ignore_index=True)
    
    if flag:
        return df_zakupki
    else:
        return N


def additional_data(df, driver, flag_return = True):

    mas_h, mas_O, mas_bool = [], [], []
    mas_len = []

    numbers = list(df['Реестровый номер закупки']) # по 223 пока вообще не ищет!!!
    fz = list(df['Закупки по'])
    df_new = df.copy()
    df_new.reset_index(inplace = True, drop = True)
    #print(df_new)

    for i in tqdm(range(len(numbers)), desc = "Adding lost information"):

        num = numbers[i]

        if fz[i] == "223-ФЗ":
            #print("Skiping 223-ФЗ")
            #print("")
            mas_h.append(np.nan)
            mas_bool.append(False)
            mas_O.append(np.nan)
            mas_len.append(0)
            continue

        driver.get(f'https://zakupki.gov.ru/epz/order/notice/ea44/view/supplier-results.html?regNumber={num[1:]}')

        #driver.refresh()

        # Дальнейшие действия, такие как извлечение HTML-кода или взаимодействие с сайтом
        page_source = driver.page_source

        # Использование BeautifulSoup для парсинга HTML-кода
        soup = bs(page_source, 'html.parser')
        #print(f"Href done: {num}")

        LST = soup.find_all("span", {"class": "section__info"})

        #cost = [str(x) for x in lst][0]
        if len(LST) != 0:
            href = [str(x) for x in LST][0]
            href = href[href.find('href="') + len('href="') : href.find('" target=')]
        else:
            href = np.nan


        if type(href) == str and href.find("sberbank-ast") != -1:
            #print("Valid Sberbank href. Printing:", href, sep = " ")
            #print("-"*20)
            mas_h.append(href)
            mas_bool.append(True)
        else:
            #print("Invalid href. Contains a file or a url of different from Sberbak-AST website.")
            #print("-"*20)
            mas_h.append(np.nan)
            mas_bool.append(False)


        driver.get(f'https://zakupki.gov.ru/epz/order/notice/ea44/view/common-info.html?regNumber={num[1:]}')

        #driver.refresh()


        page_source = driver.page_source


        soup = bs(page_source, 'html.parser')
        #print(f"ODKP done: {num}")

        lst = soup.find_all("table", {"class": "blockInfo__table tableBlock"})
        
        
        if len([x for x in lst if str(x).find("Код позиции") != -1]) == 0:
            ODKP = np.nan
            #print("yes")
            mas_O.append(ODKP)
            mas_len.append(0)
            continue
        
        lst = [x for x in lst if str(x).find("Код позиции") != -1][0]
        aa = lst.find_all("td", {"class": "tableBlock__col"})
        aa = [str(x) for x in aa]
        aa = [clear(x) for x in aa if x.find(".") != -1]

        if len(aa) == 0:
            ODKP = np.nan
        else:
            ODKP = aa[0]
        
        mas_O.append(ODKP)
        mas_len.append(len(aa))
        #print(mas_bool)
        #print(mas_O)

    df_new["Сайт закупки"] = pd.Series(mas_h)
    df_new["Bool"] = pd.Series(mas_bool)
    df_new["Классификация по ОКДП"] = pd.Series(mas_O)
    df_new["Количество товаров в лоте"] = pd.Series(mas_len)
    
    #print(df_new)
    
    df_new = df_new[df_new["Bool"] == True]
    df_new.reset_index(inplace = True, drop = True)
    del df_new["Bool"]
    
    if flag_return:
        return df_new
    else:
        return df_new.to_dict()
    
def connect_to_zakupka(website):
    ua = UserAgent()
    user_agent = ua.chrome 

    chrome_options = Options()
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-extensions')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(website)

    convert(content)
    
    with open('cookies.json', 'r') as f:
        cookies = json.load(f)

    for cookie in cookies:
        driver.add_cookie(cookie)

    driver.refresh()
    page_source = driver.page_source

    soup = bs(page_source, 'html.parser')
    
    if not is_ok(soup):
        return "update cookies"
    
    df_zakupki = pd.DataFrame(zakupka)
    dic_zak = append_zakupka(df_zakupki, soup, flag = False)
    
    return dic_zak

def is_ok(SOUP):
    soup_string = str(SOUP)
    #print(soup_string)
    return soup_string.find("GenericErrorMessageCookies") == -1

In [4]:
first_table = {'Дата принятия решения / Дата подписания письменного протокола': [],
                        'Способ определения поставщика (подрядчика, исполнителя)': [],
                        'Номер извещения': [],
                        'Идентификационный код закупки': [],
                        'Номер редакции извещения': [],'Наименование объекта закупки': [],
                        'Начальная (максимальная) цена контракта': [],
                        'Невозможно определить количество (объем) закупаемых товаров, работ, услуг': [],
                        'Дата и время начала проведения аукциона': [],
                        'Дата и время окончания проведения аукциона': []}

second_table = {"Наименование участника": [],
                        "Защищенный номер": [],
                        "Предложение о цене контракта": [],
                        "Дата и время подачи заявки": [],
                        "Дата и время подачи ценового предложения": [],
                        "Решение комиссии о соответствии или отклонении": [],
                        "Причина отклонения": [],
                        "Количество участников": [],
                        "Победитель": []}

zakupka = first_table | second_table

basing = ['Закупки по',
         'Реестровый номер закупки',
         'Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)',
         'Наименование закупки',
         'Предмет электронного аукциона (только для ПП РФ 615)',
         'Номер лота',
         'Наименование лота',
         'Начальная (максимальная) цена контракта',
         'Классификация по ОКДП',
         'Классификация по ОКПД',
         'Классификация по ОКПД2',
         'Код позиции',
         'Наименование Заказчика',
         'Организация, осуществляющая размещение ',
         'Дата размещения',
         'Дата обновления',
         'Дата начала подачи заявок',
         'Дата окончания подачи заявок',
         'Сайт закупки',
         "Количество товаров в лоте"]

base_info = {x: [] for x in basing}
auction = base_info | zakupka

### Общие таблицы с выгрузкой | Картель

In [5]:
ua = UserAgent()
user_agent = ua.chrome  # Или другой браузер по вашему выбору

# Настройки для Chrome, включающие User-Agent
chrome_options = Options()
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument('--disable-gpu')  # Отключение графики
chrome_options.add_argument('--disable-extensions')  # Отключение расширений
#chrome_options.add_argument('--headless')  # Работа без графического интерфейса

driver = webdriver.Chrome(options=chrome_options)


df = pd.read_excel(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/to_parse/1.xlsx")

In [5]:
dff = additional_data(df, driver)

Adding lost information: 100%|██████████| 11/11 [00:21<00:00,  1.96s/it]


In [7]:
for i in range(2, 24 + 1):
    df = pd.read_excel(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/to_parse/{i}.xlsx")
    dff_new = additional_data(df, driver)
    dff = pd.concat([dff, dff_new], ignore_index=True)
dff

Adding lost information: 100%|██████████| 3/3 [00:05<00:00,  1.97s/it]


,Закупки по,Реестровый номер закупки,"Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)",Наименование закупки,Предмет электронного аукциона (только для ПП РФ 615),Номер лота,Наименование лота,Начальная (максимальная) цена контракта,Классификация по ОКДП,Классификация по ОКПД,Классификация по ОКПД2,Код позиции,Наименование Заказчика,"Организация, осуществляющая размещение",Дата размещения,Дата обновления,Дата начала подачи заявок,Дата окончания подачи заявок,Сайт закупки,Количество товаров в лоте
0,44-ФЗ,№0173100012515000267,Электронный аукцион,№ 277-2015АЭ Противогаз фильтрующий ПМК-С.,NaN,NaN,NaN,1.254338e+08,33.10.16.159,NaN,NaN,NaN,МИНИСТЕРСТВО ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ ФЕДЕРАЦИИ,МИНИСТЕРСТВО ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ ФЕДЕРАЦИИ,2015-06-30,2018-02-02,2015-06-30,2015-07-16,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1
1,44-ФЗ,№0358100010015000191,Электронный аукцион,Противогаз ПМК-С,NaN,NaN,NaN,3.694627e+07,33.10.16.190,NaN,NaN,NaN,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""СЕВЕРО-КАВКАЗ...","федеральное казенное учреждение ""Северо-Кавказ...",2015-03-13,2016-09-14,2015-03-13,2015-04-13,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1
2,44-ФЗ,№0173100012514000331,Электронный аукцион,293-2014АЭ Противогаз фильтрующий ПМК-С.,NaN,NaN,NaN,1.278235e+07,25.13.60.210,NaN,NaN,NaN,МИНИСТЕРСТВО ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ ФЕДЕРАЦИИ,Министерство внутренних дел Российской Федерации,2014-08-22,2016-02-01,2014-08-22,2014-09-18,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1
3,44-ФЗ,№0362100026215000214,Электронный аукцион,Противогаз фильтрующий ПМК-С,NaN,NaN,NaN,2.224493e+07,33.10.16.159,NaN,NaN,NaN,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""УРАЛЬСКОЕ ОКР...","федеральное казенное учреждение ""Уральское ок...",2015-05-05,2015-05-21,2015-05-05,2015-05-21,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1
4,44-ФЗ,№0351100018115000065,Электронный аукцион,Противогаз фильтрующий ПМК-С,NaN,NaN,NaN,1.987226e+07,33.10.16.159,NaN,NaN,NaN,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""СИБИРСКОЕ ОКР...","ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""СИБИРСКОЕ ОКР...",2015-02-24,2015-03-26,2015-03-10,2015-03-26,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,44-ФЗ,№0372200079414000010,Электронный аукцион,Поставка средств индивидуальной зашиты,NaN,NaN,NaN,1.674996e+05,33.10.16.159,NaN,NaN,NaN,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ ...,Государственное бюджетное специальное (коррекц...,2014-07-29,2014-08-26,2014-07-30,2014-08-08,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1
752,44-ФЗ,№0372200145814000006,Электронный аукцион,Поставка противогазов для нужд Государственног...,NaN,NaN,NaN,7.280833e+04,33.10.16.191,NaN,NaN,NaN,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТ...,Государственное бюджетное дошкольное образоват...,2014-07-29,2014-08-25,2014-07-30,2014-08-07,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1
753,44-ФЗ,№0349300001714000239,Электронный аукцион,Поставка гражданских противогазов ГП-7Б,NaN,NaN,NaN,1.312000e+05,33.10.16.150,NaN,NaN,NaN,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧ...,Администрация муниципального образования город...,2014-07-30,2014-08-25,2014-07-30,2014-08-07,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1
754,44-ФЗ,№0372200043414000034,Электронный аукцион,Поставка средств индивидуальной защиты граждан...,NaN,NaN,NaN,1.458800e+04,33.10.16.153,NaN,NaN,NaN,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТ...,Государственное бюджетное дошкольное образоват...,2014-07-29,2014-08-22,2014-07-31,2014-08-11,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1


In [10]:
OKPD = sorted(list(set(list(dff["Классификация по ОКДП"])))) # конечно же, это ошибка
# имеется в виду ОКПД. Но эта ошибка тянется и далее, так что менять её затруднительно.

ua = UserAgent()
user_agent = ua.chrome  # Или другой браузер по вашему выбору

# Настройки для Chrome, включающие User-Agent
chrome_options = Options()
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument('--disable-gpu')  # Отключение графики
chrome_options.add_argument('--disable-extensions')  # Отключение расширений
#chrome_options.add_argument('--headless')  # Работа без графического интерфейса

driver = webdriver.Chrome(options=chrome_options)

for_OKPD = {}

for i in tqdm(range(len(OKPD)), "Finding description of OKPD"):
    x = OKPD[i]
    
    driver.get(f'https://lugasoft.ru/ok/okpd/2007/{x}')
    driver.refresh()

    page_source = driver.page_source

    soup = bs(page_source, 'html.parser')

    LST = str(soup.find_all("h2")[0])
    for_OKPD[(i, x)] = (clear(LST), len(dff[dff["Классификация по ОКДП"]==x]["Классификация по ОКДП"]))
    
_OKPD = {x[1]: for_OKPD[x][0] for x in for_OKPD}

mas = sorted(list(for_OKPD.items()), key = lambda x: -x[1][1])
dF = pd.DataFrame(base_info)
#print(dF)
for item in [x[0][1] for x in mas if "33.10.16" in x[0][1]]:
    dF = pd.concat([dF, dff[dff["Классификация по ОКДП"]== item]], ignore_index=True)

dF.to_excel(f'C:/Users/{DESKTOP_USER_NAME}/Desktop/root/everyone_protivogaz_cartel.xlsx', index=False)

NameError: name 'dff' is not defined

### Таблицы с участниками | Картель

In [194]:
dF = pd.read_excel(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/everyone_protivogaz_cartel.xlsx")
#dF

In [195]:
warnings.simplefilter(action='ignore', category=FutureWarning)

dic_auc = dF.to_dict()
heading = list(dic_auc.keys())
#dic_auc
mas = [{x: [dic_auc[x][i]] for x in heading} for i in range(len(dF))]

last_one = -1
fin_dic = {x: [] for x in auction}
done = set()

In [226]:
content = """[
{
    "domain": ".sberbank-ast.ru",
    "expirationDate": 1754420783,
    "hostOnly": false,
    "httpOnly": false,
    "name": "_ym_d",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": true,
    "session": false,
    "storeId": "0",
    "value": "1722884783",
    "id": 1
},
{
    "domain": ".sberbank-ast.ru",
    "expirationDate": 1722956783,
    "hostOnly": false,
    "httpOnly": false,
    "name": "_ym_isad",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": true,
    "session": false,
    "storeId": "0",
    "value": "2",
    "id": 2
},
{
    "domain": ".sberbank-ast.ru",
    "expirationDate": 1754420783,
    "hostOnly": false,
    "httpOnly": false,
    "name": "_ym_uid",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": true,
    "session": false,
    "storeId": "0",
    "value": "1722884783954240798",
    "id": 3
},
{
    "domain": "www.sberbank-ast.ru",
    "hostOnly": true,
    "httpOnly": true,
    "name": "ASP.NET_SessionId",
    "path": "/",
    "sameSite": "lax",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "1chfqif3lkblrnhsh2yygv5s",
    "id": 4
},
{
    "domain": "www.sberbank-ast.ru",
    "hostOnly": true,
    "httpOnly": false,
    "name": "BotMitigationCookie_11412016071036030678",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "\"755353001722933386Rzc9Czex29bRAWH6FVmHmqH5+Hw=\"",
    "id": 5
}
]"""

В случае прекращения работы цикла и сообщения "Update cookie!", необходимо проиграть ячейку после него, обновить куки-файлы в ячейке над этим комментарием, а после заново запустить ячейку с циклом!

In [255]:
for i in tqdm(range(len(mas))):
    auc = mas[i]
    if i <= last_one:
        continue
    if auc["Реестровый номер закупки"][0] in done:
        continue
    site = auc['Сайт закупки'][0]
    dic_zak = connect_to_zakupka(site)
    if type(dic_zak) == str:
        print("Update cookie!", last_one, sep = "\n")
        break
    
    if len(set(dic_zak["Количество участников"])) == 0:
        done.add(auc["Реестровый номер закупки"][0])
        continue
        
    amo = int(list(set(dic_zak["Количество участников"]))[0])
    #print(amo)
    prom = {x: auc[x]*amo for x in list(auc.keys())}
    
    for x in list(fin_dic.keys()):
        if x in list(prom.keys()):
            fin_dic[x] = fin_dic[x] + prom[x]
        elif x in list(dic_zak.keys()):
            #print(dic_zak[x], type(dic_zak[x]))
            fin_dic[x] = fin_dic[x] + dic_zak[x]
        else:
            print("ERROR!!!")
    last_one = i
    #print("Done:", auc["Реестровый номер закупки"][0], "\n")
    done.add(auc["Реестровый номер закупки"][0])

100%|██████████| 378/378 [04:06<00:00,  1.54it/s]


In [256]:
last_one

377

In [257]:
import IPython.display

dFF = pd.DataFrame(fin_dic)
#IPython.display.HTML(dFF.to_html())
dFF

,Закупки по,Реестровый номер закупки,"Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)",Наименование закупки,Предмет электронного аукциона (только для ПП РФ 615),Номер лота,Наименование лота,Начальная (максимальная) цена контракта,Классификация по ОКДП,Классификация по ОКПД,...,Дата и время окончания проведения аукциона,Наименование участника,Защищенный номер,Предложение о цене контракта,Дата и время подачи заявки,Дата и время подачи ценового предложения,Решение комиссии о соответствии или отклонении,Причина отклонения,Количество участников,Победитель
0,44-ФЗ,№0173100012515000267,Электронный аукцион,№ 277-2015АЭ Противогаз фильтрующий ПМК-С.,NaN,NaN,NaN,1.254338e+08,33.10.16.159,NaN,...,2015-07-27 09:45:00,"Открытое акционерное общество ""АРТИ-Завод""",NaN,0.00,NaN,NaT,Соответствует,NaN,1,True
1,44-ФЗ,№0362100026215000214,Электронный аукцион,Противогаз фильтрующий ПМК-С,NaN,NaN,NaN,2.224493e+07,33.10.16.159,NaN,...,2015-05-25 12:10:00,"Открытое акционерное общество ""АРТИ-Завод""",8992685.0,22133702.47,NaN,2015-05-25 12:00:26,Соответствует,NaN,2,True
2,44-ФЗ,№0362100026215000214,Электронный аукцион,Противогаз фильтрующий ПМК-С,NaN,NaN,NaN,2.224493e+07,33.10.16.159,NaN,...,2015-05-25 12:10:00,"Общество с ограниченной ответственностью ""СИЗ-...",1942575.0,22240215.20,NaN,2015-05-25 12:10:54,Соответствует,NaN,2,False
3,44-ФЗ,№0351100018115000065,Электронный аукцион,Противогаз фильтрующий ПМК-С,NaN,NaN,NaN,1.987226e+07,33.10.16.159,NaN,...,2015-04-03 07:45:00,"Открытое акционерное общество ""АРТИ-Завод""",1127177.0,19772893.72,NaN,2015-04-03 07:35:40,Соответствует,NaN,2,True
4,44-ФЗ,№0351100018115000065,Электронный аукцион,Противогаз фильтрующий ПМК-С,NaN,NaN,NaN,1.987226e+07,33.10.16.159,NaN,...,2015-04-03 07:45:00,"Общество с ограниченной ответственностью ""СИЗ-...",1302184.0,19870285.50,NaN,2015-04-03 07:46:05,Соответствует,NaN,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,44-ФЗ,№0301300410614000076,Электронный аукцион,Закупка средств индивидуальной защиты органов ...,NaN,NaN,NaN,3.426500e+04,33.10.16.143,NaN,...,2014-08-25 08:04:00,"Общество с ограниченной ответственностью ""Нави...",8521054.0,14634.95,NaN,2014-08-25 07:54:04,Соответствует,NaN,3,True
1062,44-ФЗ,№0301300410614000076,Электронный аукцион,Закупка средств индивидуальной защиты органов ...,NaN,NaN,NaN,3.426500e+04,33.10.16.143,NaN,...,2014-08-25 08:04:00,Общество с ограниченной ответственностью «НПК ...,8506859.0,15224.00,NaN,2014-08-25 07:55:38,Соответствует,NaN,3,False
1063,44-ФЗ,№0301300410614000076,Электронный аукцион,Закупка средств индивидуальной защиты органов ...,NaN,NaN,NaN,3.426500e+04,33.10.16.143,NaN,...,2014-08-25 08:04:00,"Общество с ограниченной ответственностью ""СИЗ-...",8513293.0,16320.49,NaN,2014-08-25 07:48:06,Соответствует,NaN,3,False
1064,44-ФЗ,№0362300030814000243,Электронный аукцион,Костюм противочумный,NaN,NaN,NaN,6.550000e+04,33.10.16.154,NaN,...,2014-11-26 12:07:00,Индивидуальный предприниматель Замана Людмила ...,4545753.0,65137.62,NaN,2014-11-26 11:57:35,Соответствует,NaN,2,True


In [259]:
dFF.to_excel(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/particip_protivogaz_cartel.xlsx", index = False)

### Общие таблицы с выгрузкой | Конкурентные

In [3]:
dff = pd.read_excel(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/everyone_protivogaz_cartel.xlsx")
dF = pd.read_excel(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/particip_protivogaz_cartel.xlsx")

In [4]:
cartel = list(set(dff["Реестровый номер закупки"]))
len(cartel)

361

ВАЖНО! Функция ниже - костыль, сделанный под конкретно эту задачу!

In [17]:
def opt_ima(X):
    if "Акционерное общество" in X or "Акционерное общество".upper() in X:
        X = X.replace("Акционерное общество", "АО")
        X = X.replace("Акционерное общество".upper(), "АО")
    name = "Закрытое акционерное общество"
    name_abr = "".join([x[0].upper() for x in name.split(" ")])
    if name in X or name.upper() in X:
        X = X.replace(name, name_abr)
        X = X.replace(name.upper(), name_abr)
    name = "Индивидуальный предприниматель"
    name_abr = "".join([x[0].upper() for x in name.split(" ")])
    if name in X or name.upper() in X:
        X = X.replace(name, name_abr)
        X = X.replace(name.upper(), name_abr)
        if X.find("(") != -1:
            X = X[:X.find("(") - 1]
        aa = X.split(" ")
        X = " ".join(aa[:2]) + " "+ aa[2][0] + ". " + aa[3][0] + "."
    name = "Индивидуальный Предприниматель"
    name_abr = "".join([x[0].upper() for x in name.split(" ")])
    if name in X or name.upper() in X:
        X = X.replace(name, name_abr)
        X = X.replace(name.upper(), name_abr)
        if X.find("(") != -1:
            X = X[:X.find("(") - 1]
        aa = X.split(" ")
        X = " ".join(aa[:2]) + " "+ aa[2][0] + ". " + aa[3][0] + "."
    name = "Общество с Ограниченной Ответственностью"
    name_abr = "".join([x[0].upper() for x in [mm for mm in name.split(" ") if len(mm) > 1]])
    if name in X or name.upper() in X:
        X = X.replace(name, name_abr)
        X = X.replace(name.upper(), name_abr)
    name = "Общество с ограниченной ответственностью"
    name_abr = "".join([x[0].upper() for x in [mm for mm in name.split(" ") if len(mm) > 1]])
    if name in X or name.upper() in X:
        X = X.replace(name, name_abr)
        X = X.replace(name.upper(), name_abr)
    name = "общество с ограниченной ответственностью"
    name_abr = "".join([x[0].upper() for x in [mm for mm in name.split(" ") if len(mm) > 1]])
    if name in X or name.upper() in X:
        X = X.replace(name, name_abr)
        X = X.replace(name.upper(), name_abr)
    name = "Открытое акционерное общество"
    name_abr = "АО"#"".join([x[0].upper() for x in name.split(" ")])
    if name in X or name.upper() in X:
        X = X.replace(name, name_abr)
        X = X.replace(name.upper(), name_abr)
    
    name = "Общество с ограниченной отвественностью"
    name_abr = "".join([x[0].upper() for x in [mm for mm in name.split(" ") if len(mm) > 1]])
    if name in X or name.upper() in X:
        X = X.replace(name, name_abr)
        X = X.replace(name.upper(), name_abr)
    name = "Общество с ограниченой ответственностью"
    name_abr = "".join([x[0].upper() for x in [mm for mm in name.split(" ") if len(mm) > 1]])
    if name in X or name.upper() in X:
        X = X.replace(name, name_abr)
        X = X.replace(name.upper(), name_abr)
    name = "Общество с ограниченной ответсвенностью"
    name_abr = "".join([x[0].upper() for x in [mm for mm in name.split(" ") if len(mm) > 1]])
    if name in X or name.upper() in X:
        X = X.replace(name, name_abr)
        X = X.replace(name.upper(), name_abr)
    
    X = X.replace("'", '"')
    X = X.replace("«", '"')
    X = X.replace("»", '"')
    X = X.replace("“", '"')
    X = X.replace("”", '"')
    
    if X.find('"') != -1 and X[-1] != '"':
        X = X + '"'
    
    if X == "Карельское республианского отделение Всероссийское добровольное пожарное общество":
        X = "КРО ВДПО"
    
    if X == 'ООО "Многопрофильная чрезвычайная служба гражданской обороны "Экран"':
        X = 'ООО "МС ГО "ЭКРАН"'
    
    if X == 'ООО "Новгородский центр пожарной безопасности плюс"':
        X = 'ООО "НЦПБ+"'
    
    if X == 'ООО "Оптово-материальная часть по снабжению "Резерв"':
        X = 'ООО "ОМЧС "Резерв"'
    
    if X == 'ООО Вариант М':
        X = 'ООО "Вариант-М"'
    
    if X == 'ООО Внешнеэкономическая Дистрибьюторская Фирма "Акцепт"':
        X = 'ООО ВДФ "АКЦЕПТ"'
    
    if X == 'ООО МедАгро':
        X = 'ООО "МедАгро"'
    
    if X == 'АО "Электростальский химико-механический завод имени Н.Д.Зелинского"':
        X = 'АО "ЭХМЗ им Н.Д. Зелинского"'
    
    if X == 'ООО Образовательный проект':
        X = 'ООО "Образовательный проект"'
    
    return X

def check(X):
    if X.find('"') == -1:
        return X
    if X[X.find('"') - 1] != " ":
        return X[:X.find('"')] + " " + X[X.find('"'):]
    else:
        return X

def control(mas):
    individ = list(set([x.lower() for x in mas]))
    off = dict()
    for x in mas:
        if not(x.lower() in off) and x.lower() in individ:
            off[x.lower()] = x
            #print(x)
    mas_new = [off[x.lower()] for x in mas]
    return mas_new

In [18]:
imena = list(dF["Наименование участника"])

opt_imena = [check(opt_ima(x)) for x in imena]

opt_imena = control(opt_imena)

In [23]:
dF["Наименование участника"] = pd.Series(opt_imena)

In [256]:
#dF.iloc[[j for j, x in enumerate(opt_imena) if x == 'ООО "МедАгро"']]

file = open(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/carteling.txt", "r+", encoding = "utf-8")
carteling = file.readlines()
file.close()

carteling = [x.strip().split(", ") for x in carteling]
carteling = [(x[0], x[1][x[1].find(" ") + 1: ]) for x in carteling]
carteling = dict(carteling)

for x in list(carteling.keys()):
    if not(x in set(opt_imena)):
        print(x)

file = open(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/competing.txt", "r+", encoding = "utf-8")
competing = file.readlines()
file.close()

competing = [x.strip().split(", ") for x in competing]
competing = [(x[0], x[1][x[1].find(" ") + 1: ]) for x in competing]
competing = dict(competing)

for x in list(competing.keys()):
    if not(x in set(opt_imena)):
        print(x)

#set(competing) | set(carteling) == set(opt_imena)

Ссылки для скачивания CSV файлов, создающиеся по шаблону: в самой ссылке уже записаны даты, между которыми должны быть закупки (берутся из решения ФАС), ФЗ для процедуры, площадка проведения закупки (Сбербанк-АСТ) и прочее. Словом, если открыть закомментированную ссылку, то на правой панели можно все увидеть самому и поменять под свои нужды.

In [261]:
# ТИПОВАЯ ССЫЛКА ДЛЯ ВЫГРУЗКИ CSV
# https://zakupki.gov.ru/epz/order/extendedsearch/results.html?morphology=on&pageNumber=1&sortDirection=false&recordsPerPage=_50&showLotsInfoHidden=true&savedSearchSettingsIdHidden=setting_order_lvrncsp3&sortBy=UPDATE_DATE&fz44=on&fz94=on&af=on&ca=on&pc=on&pa=on&etp_7360769=on&etp=7360769&currencyIdGeneral=-1&publishDateFrom=01.02.2014&publishDateTo=31.12.2015&participantName=7817043607&gws=Выберите+тип+закупки&OrderPlacementSmallBusinessSubject=on&OrderPlacementRnpData=on&OrderPlacementExecutionRequirement=on&orderPlacement94_0=0&orderPlacement94_1=0&orderPlacement94_2=0

for i, x in enumerate(competing):
    if i % 3 == 0:
        #print("="*100, "\n")
        continue
    href = f'https://zakupki.gov.ru/epz/order/extendedsearch/results.html?morphology=on&pageNumber=1&sortDirection=false&recordsPerPage=_50&showLotsInfoHidden=true&savedSearchSettingsIdHidden=setting_order_lvrncsp3&sortBy=UPDATE_DATE&fz44=on&fz94=on&af=on&ca=on&pc=on&pa=on&etp_7360769=on&etp=7360769&currencyIdGeneral=-1&publishDateFrom=01.02.2014&publishDateTo=31.12.2015&participantName={competing[x]}&gws=Выберите+тип+закупки&OrderPlacementSmallBusinessSubject=on&OrderPlacementRnpData=on&OrderPlacementExecutionRequirement=on&orderPlacement94_0=0&orderPlacement94_1=0&orderPlacement94_2=0'
    print(href, "\n")

Код для сохранения csv файлов в формате txt

In [299]:
import os

i = 1

os.chdir(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/to_convert/")

for filename in os.listdir(r"C:/Users/Roman/Desktop/root/to_convert"):
    if filename.endswith(".txt"):
        print(filename)
        os.rename(filename, f"{i}.txt")
    i += 1

'\nimport os\n\ni = 1\n\nos.chdir("C:/Users/Admin/Desktop/to_convert/")\n\nfor filename in os.listdir(r"C:/Users/Admin/Desktop/to_convert"):\n    if filename.endswith(".txt"):\n        print(filename)\n        os.rename(filename, f"{i}.txt")\n    i += 1\n    \n'

In [316]:
ua = UserAgent()
user_agent = ua.chrome  # Или другой браузер по вашему выбору

# Настройки для Chrome, включающие User-Agent
chrome_options = Options()
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument('--disable-gpu')  # Отключение графики
chrome_options.add_argument('--disable-extensions')  # Отключение расширений
#chrome_options.add_argument('--headless')  # Работа без графического интерфейса

driver = webdriver.Chrome(options=chrome_options)


ff = pd.read_csv(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/to_convert/1.txt", delimiter = ";", encoding="cp1251")

In [317]:
mas_df = additional_data(ff, driver) #, flag_return = False)

Adding lost information: 100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


In [318]:
mas_df

,Закупки по,Реестровый номер закупки,"Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)",Наименование закупки,Предмет электронного аукциона (только для ПП РФ 615),Номер лота,Наименование лота,Начальная (максимальная) цена контракта,Классификация по ОКДП,Классификация по ОКПД,Классификация по ОКПД2,Код позиции,Наименование Заказчика,"Организация, осуществляющая размещение",Дата размещения,Дата обновления,Дата начала подачи заявок,Дата окончания подачи заявок,Сайт закупки,Количество товаров в лоте
0,44-ФЗ,№0373100093515000006,Электронный аукцион,(1-эа/15) Оказание услуг по техническому обслу...,NaN,NaN,NaN,1227272.73,70.32.13.130,NaN,NaN,NaN,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,федеральное казенное учреждение здравоохранени...,26.01.2015,25.03.2015,16.03.2015,24.03.2015,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1


Код ниже работает ~15 часов!

In [319]:
for i in range(2, 106 + 1):
    if i == 10:
        continue
    df = pd.read_csv(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/to_convert/{i}.txt", delimiter = ";", encoding="cp1251")
    dff_new = additional_data(df, driver)
    mas_df = pd.concat([mas_df, dff_new], ignore_index=True)

Adding lost information: 100%|██████████| 31/31 [01:12<00:00,  2.33s/it]


In [308]:
mas_df

,Закупки по,Реестровый номер закупки,"Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)",Наименование закупки,Предмет электронного аукциона (только для ПП РФ 615),Номер лота,Наименование лота,Начальная (максимальная) цена контракта,Классификация по ОКДП,Классификация по ОКПД,Классификация по ОКПД2,Код позиции,Наименование Заказчика,"Организация, осуществляющая размещение",Дата размещения,Дата обновления,Дата начала подачи заявок,Дата окончания подачи заявок,Сайт закупки,Количество товаров в лоте
0,44-ФЗ,№0373100093515000006,Электронный аукцион,(1-эа/15) Оказание услуг по техническому обслу...,NaN,NaN,NaN,1227272.73,70.32.13.130,NaN,NaN,NaN,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,федеральное казенное учреждение здравоохранени...,26.01.2015,25.03.2015,16.03.2015,24.03.2015,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1.0
1,44-ФЗ,№0349300031314000001,Электронный аукцион,Ограждение и сопутствующие материалы,NaN,NaN,NaN,895173.80,NaN,NaN,NaN,NaN,МУНИЦИПАЛЬНОЕ АВТОНОМНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТЕ...,Муниципальное бюджетное дошкольное образовател...,11.07.2014,20.01.2017,14.07.2014,21.07.2014,NaN,NaN
2,44-ФЗ,№0349100016715000044,Электронный аукцион,поставка товаров хозяйственного назначения (са...,NaN,NaN,NaN,35871.93,NaN,NaN,NaN,NaN,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,22.04.2015,21.07.2016,22.04.2015,05.05.2015,NaN,NaN
3,44-ФЗ,№0349100004614000187,Электронный аукцион,Поставка хозяйственных товаров (посуда) в 2014...,NaN,NaN,NaN,102250.00,NaN,NaN,NaN,NaN,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,Федеральное государственное бюджетное учрежден...,11.09.2014,01.02.2016,12.09.2014,19.09.2014,NaN,NaN
4,44-ФЗ,№0149100002314000264,Электронный аукцион,Поставка охлаждающей жидкости Тосол А-40 (тем...,NaN,NaN,NaN,143400.00,NaN,NaN,NaN,NaN,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕН...,федеральное государственное казенное учреждени...,04.09.2014,01.02.2016,04.09.2014,17.09.2014,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14385,44-ФЗ,№0345300021114000015,Электронный аукцион,Поставка детского игрового учебного оборудован...,NaN,NaN,NaN,359000.00,NaN,NaN,NaN,NaN,МУНИЦИПАЛЬНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТЕЛЬНОЕ БЮДЖЕ...,Муниципальное дошкольное образовательное бюдже...,23.06.2014,01.07.2014,24.06.2014,01.07.2014,NaN,NaN
14386,44-ФЗ,№0345300017514000019,Электронный аукцион,приобретение детского игрового учебного оборуд...,NaN,NaN,NaN,89000.02,NaN,NaN,NaN,NaN,МУНИЦИПАЛЬНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТЕЛЬНОЕ БЮДЖЕ...,Муниципальное дошкольное образовательное бюдже...,21.06.2014,30.06.2014,23.06.2014,30.06.2014,NaN,NaN
14387,44-ФЗ,№0372200029014000003,Электронный аукцион,Поставка одежды с логотипом учреждения для ГБО...,NaN,NaN,NaN,71000.00,NaN,NaN,NaN,NaN,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ДОПОЛНИТЕ...,государственное бюджетное образовательное учре...,30.05.2014,16.06.2014,30.05.2014,16.06.2014,NaN,NaN
14388,44-ФЗ,№0372200029014000004,Электронный аукцион,Поставка канцелярских товаров с логотипом учре...,NaN,NaN,NaN,91784.50,NaN,NaN,NaN,NaN,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ДОПОЛНИТЕ...,государственное бюджетное образовательное учре...,30.05.2014,16.06.2014,30.05.2014,16.06.2014,NaN,NaN


In [320]:
mas_df

,Закупки по,Реестровый номер закупки,"Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)",Наименование закупки,Предмет электронного аукциона (только для ПП РФ 615),Номер лота,Наименование лота,Начальная (максимальная) цена контракта,Классификация по ОКДП,Классификация по ОКПД,Классификация по ОКПД2,Код позиции,Наименование Заказчика,"Организация, осуществляющая размещение",Дата размещения,Дата обновления,Дата начала подачи заявок,Дата окончания подачи заявок,Сайт закупки,Количество товаров в лоте
0,44-ФЗ,№0373100093515000006,Электронный аукцион,(1-эа/15) Оказание услуг по техническому обслу...,NaN,NaN,NaN,1227272.73,70.32.13.130,NaN,NaN,NaN,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,федеральное казенное учреждение здравоохранени...,26.01.2015,25.03.2015,16.03.2015,24.03.2015,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1
1,44-ФЗ,№0349100016715000044,Электронный аукцион,поставка товаров хозяйственного назначения (са...,NaN,NaN,NaN,35871.93,26.22.10.117,NaN,NaN,NaN,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,22.04.2015,21.07.2016,22.04.2015,05.05.2015,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,10
2,44-ФЗ,№0349100004614000187,Электронный аукцион,Поставка хозяйственных товаров (посуда) в 2014...,NaN,NaN,NaN,102250.00,28.61.14.119,NaN,NaN,NaN,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,Федеральное государственное бюджетное учрежден...,11.09.2014,01.02.2016,12.09.2014,19.09.2014,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,10
3,44-ФЗ,№0149100002314000264,Электронный аукцион,Поставка охлаждающей жидкости Тосол А-40 (тем...,NaN,NaN,NaN,143400.00,24.66.31.141,NaN,NaN,NaN,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕН...,федеральное государственное казенное учреждени...,04.09.2014,01.02.2016,04.09.2014,17.09.2014,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1
4,44-ФЗ,№0349100004614000176,Электронный аукцион,Поставка мягкого инвентаря в 2014 году,NaN,NaN,NaN,608884.10,17.40.12.211,NaN,NaN,NaN,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,Федеральное государственное бюджетное учрежден...,14.08.2014,01.02.2016,15.08.2014,12.09.2014,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12809,44-ФЗ,№0301300021415000003,Электронный аукцион,Закупка расходного материала (перчатки),NaN,NaN,NaN,135699.00,25.13.60,NaN,NaN,NaN,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,Государственное бюджетное учреждение здравоохр...,05.03.2015,06.04.2015,05.03.2015,18.03.2015,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,8
12810,44-ФЗ,№0306200018215000032,Электронный аукцион,Поставка перчаток медицинских,NaN,NaN,NaN,116389.30,25.13.60.110,NaN,NaN,NaN,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,Государственное бюджетное учреждение здравоохр...,11.02.2015,06.04.2015,12.02.2015,19.02.2015,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,1
12811,44-ФЗ,№0140200000815000819,Электронный аукцион,Поставка перчаток,NaN,NaN,NaN,212110.00,25.13.60.110,NaN,NaN,NaN,КИРОВСКОЕ ОБЛАСТНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,ДЕПАРТАМЕНТ ГОСУДАРСТВЕННЫХ ЗАКУПОК КИРОВСКОЙ ...,24.03.2015,06.04.2015,27.03.2015,06.04.2015,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,10
12812,44-ФЗ,№0136200003615001001,Электронный аукцион,Поставка расходных медицинских изделий для нуж...,NaN,NaN,NaN,263046.00,25.13.60.110,NaN,NaN,NaN,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,Министерство имущественных и земельных отношен...,27.03.2015,06.04.2015,27.03.2015,06.04.2015,http://www.sberbank-ast.ru/ViewDocument.aspx?i...,20


In [321]:
mas_df.to_excel(f'C:/Users/{DESKTOP_USER_NAME}/Desktop/root/probably_competing.xlsx', index=False)

In [324]:
mas_dF = pd.DataFrame(base_info)
OKDP = list(mas_df["Классификация по ОКДП"])
number = list(mas_df["Реестровый номер закупки"])
cartel
#print(dF)
for item in set(OKDP):
    if "33.10.16" in item:
        mas_dF = pd.concat([mas_dF, mas_df[mas_df["Классификация по ОКДП"]== item]], ignore_index=True)

In [325]:
mas_dF = mas_dF.drop_duplicates()
len(mas_dF)

831

In [326]:
DF = pd.DataFrame(base_info)

for item in set(number):
    if not(item in cartel):
        DF = pd.concat([DF, mas_dF[mas_dF["Реестровый номер закупки"]== item]], ignore_index=True)

In [327]:
DF.to_excel(f'C:/Users/{DESKTOP_USER_NAME}/Desktop/root/probably_competing_protivogaz.xlsx', index=False)
len(DF)

529

### Таблицы с участниками | Конкурентные

In [358]:
warnings.simplefilter(action='ignore', category=FutureWarning)

dic_auc = DF.to_dict()
heading = list(dic_auc.keys())
#dic_auc
mas = [{x: [dic_auc[x][i]] for x in heading} for i in range(len(DF))]

last_one = -1
fin_dic = {x: [] for x in auction}
done = set()

In [386]:
content = """[
{
    "domain": ".sberbank-ast.ru",
    "expirationDate": 1731777826,
    "hostOnly": false,
    "httpOnly": false,
    "name": "_ym_d",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": true,
    "session": false,
    "storeId": "0",
    "value": "1700241827",
    "id": 1
},
{
    "domain": ".sberbank-ast.ru",
    "expirationDate": 1715064643,
    "hostOnly": false,
    "httpOnly": false,
    "name": "_ym_isad",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": true,
    "session": false,
    "storeId": "0",
    "value": "2",
    "id": 2
},
{
    "domain": ".sberbank-ast.ru",
    "expirationDate": 1731777826,
    "hostOnly": false,
    "httpOnly": false,
    "name": "_ym_uid",
    "path": "/",
    "sameSite": "no_restriction",
    "secure": true,
    "session": false,
    "storeId": "0",
    "value": "1700241827330533764",
    "id": 3
},
{
    "domain": "www.sberbank-ast.ru",
    "hostOnly": true,
    "httpOnly": true,
    "name": "ASP.NET_SessionId",
    "path": "/",
    "sameSite": "lax",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "zkxhxcbvikc1j3mpdtf2ekhb",
    "id": 4
},
{
    "domain": "www.sberbank-ast.ru",
    "hostOnly": true,
    "httpOnly": false,
    "name": "BotMitigationCookie_11412016071036030678",
    "path": "/",
    "sameSite": "unspecified",
    "secure": false,
    "session": true,
    "storeId": "0",
    "value": "\"159365001715019133tPhw7j+yhSTvOAoag9qbUCB8hsU=\"",
    "id": 5
}
]"""

In [345]:
os.chdir(r"C:\Users\\" + DESKTOP_USER_NAME) # {DESKTOP_USER_NAME}

In [391]:
for i in tqdm(range(len(mas))):
    auc = mas[i]
    if i <= last_one:
        continue
    if auc["Реестровый номер закупки"][0] in done:
        continue
    site = auc['Сайт закупки'][0]
    dic_zak = connect_to_zakupka(site)
    if type(dic_zak) == str:
        print("Update cookie!", last_one, sep = "\n")
        break
    
    if len(set(dic_zak["Количество участников"])) == 0:
        done.add(auc["Реестровый номер закупки"][0])
        continue
        
    amo = int(list(set(dic_zak["Количество участников"]))[0])
    #print(amo)
    prom = {x: auc[x]*amo for x in list(auc.keys())}
    
    for x in list(fin_dic.keys()):
        if x in list(prom.keys()):
            fin_dic[x] = fin_dic[x] + prom[x]
        elif x in list(dic_zak.keys()):
            #print(dic_zak[x], type(dic_zak[x]))
            fin_dic[x] = fin_dic[x] + dic_zak[x]
        else:
            print("ERROR!!!")
    last_one = i
    #print("Done:", auc["Реестровый номер закупки"][0], "\n")
    done.add(auc["Реестровый номер закупки"][0])

100%|██████████| 529/529 [01:03<00:00,  8.32it/s]


In [392]:
last_one

528

In [393]:
import IPython.display

DFC = pd.DataFrame(fin_dic)
#IPython.display.HTML(dFF.to_html())
DFC

,Закупки по,Реестровый номер закупки,"Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)",Наименование закупки,Предмет электронного аукциона (только для ПП РФ 615),Номер лота,Наименование лота,Начальная (максимальная) цена контракта,Классификация по ОКДП,Классификация по ОКПД,...,Дата и время окончания проведения аукциона,Наименование участника,Защищенный номер,Предложение о цене контракта,Дата и время подачи заявки,Дата и время подачи ценового предложения,Решение комиссии о соответствии или отклонении,Причина отклонения,Количество участников,Победитель
0,44-ФЗ,№0336100005415000019,Электронный аукцион,На поставку средств индивидуальной защиты (про...,NaN,NaN,NaN,211653.20,33.10.16.159,NaN,...,2015-07-27 14:24:00,Общество с ограниченной ответственностью «СИЗ ...,NaN,94836.18,NaN,2015-07-27 14:14:27,Соответствует,NaN,2,True
1,44-ФЗ,№0336100005415000019,Электронный аукцион,На поставку средств индивидуальной защиты (про...,NaN,NaN,NaN,211653.20,33.10.16.159,NaN,...,2015-07-27 14:24:00,Общество с ограниченной ответственностью «Спец...,NaN,95894.45,NaN,2015-07-27 14:14:10,Соответствует,NaN,2,False
2,44-ФЗ,№0372200216815000013,Электронный аукцион,Поставка средств индивидуальной защиты для Гос...,NaN,NaN,NaN,42649.95,33.10.16.191,NaN,...,2016-01-22 12:00:00,"Общество с ограниченной ответственностью ""Торг...",NaN,0.00,NaN,NaT,Соответствует,NaN,1,True
3,44-ФЗ,№0346200000215000066,Электронный аукцион,Аукцион в электронной форме № 15-ЭА-15РМ на за...,NaN,NaN,NaN,302865.48,33.10.16.150,NaN,...,2015-08-17 10:04:00,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МЕДЛ...",NaN,151200.00,NaN,2015-08-17 09:54:56,Соответствует,NaN,5,True
4,44-ФЗ,№0346200000215000066,Электронный аукцион,Аукцион в электронной форме № 15-ЭА-15РМ на за...,NaN,NaN,NaN,302865.48,33.10.16.150,NaN,...,2015-08-17 10:04:00,Общество с ограниченной ответственностью МедАгро,NaN,152785.67,NaN,2015-08-17 09:54:26,Соответствует,NaN,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,44-ФЗ,№0301100003715000148,Электронный аукцион,ЭА 145/15 Поставка противогазов,NaN,NaN,NaN,283715.70,33.10.16.150,NaN,...,2015-11-27 10:13:00,"Общество с ограниченной ответственностью ""Торг...",NaN,224135.35,NaN,2015-11-27 10:02:32,Соответствует,NaN,5,False
1273,44-ФЗ,№0301100003715000148,Электронный аукцион,ЭА 145/15 Поставка противогазов,NaN,NaN,NaN,283715.70,33.10.16.150,NaN,...,2015-11-27 10:13:00,"Общество с ограниченной ответственностью ""КОМП...",NaN,265274.17,NaN,2015-11-27 09:44:15,Соответствует,NaN,5,False
1274,44-ФЗ,№0301100003715000148,Электронный аукцион,ЭА 145/15 Поставка противогазов,NaN,NaN,NaN,283715.70,33.10.16.150,NaN,...,2015-11-27 10:13:00,Общество с ограниченной ответственностью Торго...,NaN,268111.33,NaN,2015-11-27 09:35:32,Отклонена,NaN,5,False
1275,44-ФЗ,№0301100003715000148,Электронный аукцион,ЭА 145/15 Поставка противогазов,NaN,NaN,NaN,283715.70,33.10.16.150,NaN,...,2015-11-27 10:13:00,"Общество с ограниченной ответственностью ""Башс...",NaN,282297.12,NaN,2015-11-27 09:35:11,Соответствует,NaN,5,False


In [394]:
DFC.to_excel(f"C:/Users/{DESKTOP_USER_NAME}/Desktop/root/particip_probably_competing_protivogaz.xlsx", index = False)